# Cross Validation Nedir

** Cross validation amacı modelin genelleme performansını tahmin etmektir.Veriyi defalarca farklı şekillerde bölüp modelin başarısını tekrar tekrar test eder


* 1-Veriyi birden fazla defa eğitim ve test setlerine böleriz
* 2-Her bölmede model eğitilir
* 3-Her test seti için hata hesaplanır(mse gibi)
* 4-Ortalama hata hesaplanır

** Parametredeki shuffle=True olduğunda

* Veriler rastgele karışır sonra K parçaya bölünür
* Bu sayede her fold daha rastgele ve çeşitli örnekler içerir
* Veride belirli bir sıralama varsa çok önemlidir

## K-Fold Cross Validation

* Veri K parçaya bölünür.Her bir parça bir kez test verisi olur, geri kalan K-1 parça eğitim için kullanılır
* Avantajı veriyi daha verimli kullanır ve genelleme hatasını daha güvenilir ölçer

* Dezavantajı veri dengesizse bazı foldlarda sınıf dağılımı bozulabilir

In [1]:
#Örnek kullanım
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True,random_state=31)

## Stratified(katmanlı) K-Fold Cross Validation
* Kfoldun sınıf dengesini koruyan versiyonudur.Her fold orijinal veri kümesindeki sınıf oranlarını yansıtır
* Sınıflar dengesiz ise kullanılır(örn %90 class A,yüzde 10 class B)

In [ ]:
#Örnek kullanım
from sklearn.model_selection import StratifiedKFold
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=31)

## Leave-One-Out Cross Validation(LOOCV)

* Verideki her örnek sırayla test verisi olur.Yani N örnek var ise N kez model eğitilir
* Az veri varsa işe yarar
* Çok yavaştır büyük veri setlerinde pratik değildir kullanmak

In [2]:
#Örnek kullanım
from sklearn.model_selection import LeaveOneOut
loo=LeaveOneOut()

## Leave-P-Out Cross Validation(LPOCV)
* LOO'nun genelleştirilmiş verisyonudur.Her test seti p örnek içerir.Tüm olası kombinasyonlar denenir
* (n/p) adet kombinasyon oluşur
* Kombinasyon sayısı çok fazla olacağı için NEREDEYSE HİÇ KULLANILMAZ

## Repeated K-Fold
* K-Fold işlemi birden fazla kez farklı shufflarla tekrarlanır.Böylece sonuçlar daha kararlı hale gelir ve rastgeleliğin etkisi azalır

In [3]:
#Örnek kullanım
from sklearn.model_selection import RepeatedKFold
rkf=RepeatedKFold(n_splits=5,n_repeats=10,random_state=31)

## TimeSeriesSplit(Zaman Serileri İçin)
* Zaman sıralı verilerde geçmiş verilerle eğitiliğ gelecekle test yapılır.Shuffle yapılmaz.Foldlar zamansal olarak ilerle
* Avantajı-->Gerçekci zaman tabanlı tahmin senaryoları
* Dezavantajı-->Eğitim seti küçükkken hata yüksek olabilir

In [5]:
#Örnek kullanım
from sklearn.model_selection import TimeSeriesSplit
tss=TimeSeriesSplit(n_splits=5)

### Hangi CV yöntemi ne zaman kullanılır

* KFOLD--> genel amaçlı,dengeli veri setlerinde
* Stratified KFOLD--> Sınıflar dengesizse
* LOOCV--> Az veri varsa,yüksek doğruluk isteniyorsa
* Repeated KFOLD--> Sonuçları daha kararlı hale getirmek istiyorsak
* TimeSeriesSplit--> Zaman serileri ile çalışıyorsak

# Örnek Uygulama

In [14]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import(KFold,StratifiedKFold,LeaveOneOut,
                                    LeavePOut,RepeatedKFold,TimeSeriesSplit,
                                    cross_val_score)

In [10]:
random_state=31
#yapay verileri üretme
X,y=make_classification(n_samples=20,n_features=5,n_informative=3,
                        n_redundant=0,n_classes=2,random_state=random_state)

model=RandomForestClassifier(random_state=random_state)

## 1-) KFOLD Cross Validation

In [13]:
kf=KFold(n_splits=5,shuffle=True,random_state=random_state)
scores=cross_val_score(model,X,y,cv=kf)
print("K-Fold Accuracy",scores)
print("Ortalama Accuracy",scores.mean())

K-Fold Accuracy [0.5 0.5 0.5 0.5 0.5]
Ortalama Accuracy 0.5


## 2-) Stratified KFOLD Cross Validation

In [16]:
skf=StratifiedKFold(n_splits=5,random_state=random_state,shuffle=True)
scores=cross_val_score(model,X,y,cv=skf)
print("Stratified K-Fold Accuracy",scores)
print("Ortalama Accuracy",scores.mean())

Stratified K-Fold Accuracy [1.   0.75 0.25 1.   0.5 ]
Ortalama Accuracy 0.7


## 3-) Leave-One-Out Cross Validation(LOOCV)

In [18]:
loo=LeaveOneOut() #20 örnek olduğu için 20 kez model eğitilecek
scores=cross_val_score(model,X,y,cv=loo)

print("LOOCV Accuracy",scores)
print("Ortalama Accuracy",scores.mean())

LOOCV Accuracy [0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1.]
Ortalama Accuracy 0.55


## 4-) Leave-P-Out Cross Validation(LPOCV)

In [19]:
lpo=LeavePOut(p=2) #20*19 / 2 tane kombinasyon olcak yani 190 tane
scores=cross_val_score(model,X,y,cv=lpo)

print("LPOCV Accuracy",scores)
print("Ortalama Accuracy",scores.mean())

LPOCV Accuracy [0.5 0.5 0.  0.5 0.5 0.  0.5 0.5 0.5 0.  0.  0.  0.5 0.5 0.  0.5 0.5 0.
 0.5 1.  0.5 0.5 1.  0.5 1.  1.  1.  0.5 0.5 0.5 1.  1.  0.5 0.5 1.  0.5
 1.  0.5 1.  1.  0.5 1.  1.  1.  0.5 0.5 1.  1.  0.5 0.5 1.  0.  0.5 1.
 0.  0.  0.  0.5 0.5 0.5 0.  0.  0.5 0.5 0.5 0.  0.5 0.5 0.  0.5 0.  0.
 0.5 1.  0.5 0.  0.5 1.  1.  1.  0.5 1.  1.  0.5 0.5 0.5 1.  1.  0.5 0.5
 0.5 1.  1.  1.  0.5 1.  0.  0.5 1.  0.5 0.5 0.5 0.  0.5 0.  0.5 0.5 0.
 0.5 0.5 0.  0.5 1.  1.  0.  0.5 1.  0.5 1.  0.5 1.  1.  0.  0.  1.  0.5
 0.5 1.  1.  1.  0.5 1.  1.  0.5 1.  0.5 0.5 1.  1.  1.  0.5 1.  1.  0.5
 1.  0.  0.5 0.5 0.5 0.  0.5 0.5 0.  0.5 0.5 0.5 0.5 0.  0.5 0.5 0.  0.5
 1.  0.5 0.5 0.5 1.  0.5 1.  1.  0.5 1.  1.  0.5 0.5 0.5 1.  1.  1.  1.
 0.5 0.5 0.  0.5 1.  0.5 1.  1.  1.  0.5]
Ortalama Accuracy 0.5842105263157895


## 5-) Repeated KFOLD Cross Validation

In [22]:
rkf=RepeatedKFold(n_splits=5,n_repeats=3,random_state=random_state)
scores=cross_val_score(model,X,y,cv=rkf)

print("Repeated K-Fold Accuracy",scores)
print("Ortalama Accuracy",scores.mean())

Repeated K-Fold Accuracy [0.5  0.5  0.5  0.5  0.5  0.75 0.5  0.75 0.5  0.5  0.5  0.25 0.75 0.75
 0.25]
Ortalama Accuracy 0.5333333333333333


## 6-) TimeSeriesSplit

In [23]:
tscv=TimeSeriesSplit(n_splits=5)
scores=cross_val_score(model,X,y,cv=tscv)

print("TimeSeriesSplit Accuracy",scores)
print("Ortalama Accuracy",scores.mean())

TimeSeriesSplit Accuracy [0.66666667 0.33333333 0.66666667 0.66666667 1.        ]
Ortalama Accuracy 0.6666666666666666
